Doubly Robust法（DR法）による因果推論の実装

簡単に言えば，回帰モデルとITPW法のハイブリット

基本的な考え方は，ITPW法だと以下式で，あるiさんについて片方のケースしか考えられておらずもったいないため，  
ATE_1_i = Y/Z_pre[:, 1]*Z + (1-Z/Z_pre[:, 1])*Y_1  
反実仮想となるもう片方側を回帰分析で求めることで両側面でITPWをしようということ．

# !!重要!!

線形回帰分析やロジスティック回帰を用いるITPWはあくまで，ZやYがXの線形和で表される場合に限る．そうでない場合は性能が悪化するので注意が必要．

# 自前データの作成

In [2]:
# 乱数シード
import random
import numpy as np

np.random.seed(1234)
random.seed(1234)


# 使用するパッケージ（ライブラリと関数）を定義
# 標準正規分布の生成用
from numpy.random import *
# グラフの描画用
import matplotlib.pyplot as plt
# SciPy 平均0、分散1に正規化（標準化）関数
import scipy.stats
# シグモイド関数をimport
from scipy.special import expit
# その他
import pandas as pd


# サンプル数
num_data = 200

# 性別 / 年齢（一様分布）
# age
x_1 = randint(15, 76, num_data) 
# 性別（0: 女性，1: 男性）
x_2 = randint(0, 2, num_data)


# テレビCMをみたかどうか
# ノイズの生成
e_z = randn(num_data)
# シグモイド関数に入れる線形関数・・・（1）
z_base = x_1 + (1 - x_2)*10 - 40 + 5*e_z
# シグモイド関数の計算
z_prob = expit(0.1*z_base)
# テレビCMを見たかどうか（0: 見ていない，1: 見た）
Z = np.array([])
# （1）より年配の女性の方が見やすくなる
for i in range(num_data):
    Z_i = np.random.choice(2, size=1, p=[1-z_prob[i], z_prob[i]])[0]
    Z = np.append(Z, Z_i)
    

# 購入量Y
# ノイズの生成
e_y = randn(num_data)
# 若年層で，男性で，テレビを見ていると購入が多くなる
Y = -x_1 + 30*x_2 + 10*Z + 80 + 10*e_y 

# 作成データ

In [3]:
df = pd.DataFrame({
    'age': x_1,
    'sex': x_2,
    'cm': Z,
    'amount': Y
})

In [4]:
df.head()

,age,sex,cm,amount
0,62,0,1.0,24.464285
1,34,0,0.0,45.693411
2,53,1,1.0,64.998281
3,68,1,1.0,47.186898
4,27,1,0.0,100.114260


## CM別平均

In [5]:
print(df[df['cm']==1.0].mean())
print('------')
print(df[df['cm']==0.0].mean())

age       55.836066
sex        0.483607
cm         1.000000
amount    49.711478
dtype: float64
------
age       32.141026
sex        0.692308
cm         0.000000
amount    68.827143
dtype: float64


# DR法

## 回帰分析の実施

In [6]:
from sklearn.linear_model import LinearRegression

# 説明変数
X = df[['age', 'sex', 'cm']]

# 目的変数
y = df['amount']

# 回帰
reg2 = LinearRegression().fit(X, y)

# Z=0（cmを見ていない）場合のデータ
X_0 = X.copy()
X_0['cm'] = 0
Y_0 = reg2.predict(X_0)

# Z=1（cmを見ていた）場合のデータ
X_1 = X.copy()
X_1['cm'] = 1
Y_1 = reg2.predict(X_1)

## 傾向スコアの推定

In [7]:
from sklearn.linear_model import LogisticRegression

# 説明変数
X = df[['age', 'sex']]

# 目的変数
Z = df['cm']

# 回帰
reg = LogisticRegression().fit(X, Z)

# 傾向スコア
Z_pre = reg.predict_proba(X)
print(Z_pre[0:5]) # 5人くらいの結果

[[0.04002323 0.95997677]
 [0.44525168 0.55474832]
 [0.30065918 0.69934082]
 [0.08101946 0.91898054]
 [0.87013558 0.12986442]]


## 平均処置効果ATE

In [9]:
ATE_1_i = Y/Z_pre[:, 1]*Z + (1-Z/Z_pre[:, 1])*Y_1
ATE_0_i = Y/Z_pre[:, 0]*(1-Z) + (1-(1-Z)/Z_pre[:, 0])*Y_0
ATE = 1/len(Y)*(ATE_1_i-ATE_0_i).sum()
print('推定されるATE', ATE)

推定されるATE 9.75277505424846
